In [1]:
#import tools, load required data
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import os
pd.set_option("display.max_rows",300)
pd.set_option("display.max_columns",100)

In [2]:
#load all files
preDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_before_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
orderDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_at_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
medsBeforeOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_before_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
medsAtOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_at_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
dischargeMeds = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_after_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
orderInfo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/Order_info_PN.csv")
problem_list = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Problem_List.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "DX_time", "PP_YN", "Activity"))
demo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Demographic.csv")
social = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/social.csv")
dischargeDiag = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_diag.csv")
dischargeMeds = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_meds.csv", encoding = "ISO-8859-1")
gramSnomed = pd.read_csv("/home/ucsdantibiogram/notebooks/data/organism_SNOMED_gram.csv")


/home/cuda/anaconda3/envs/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Pre Process OrderInfo

In [3]:
orderInfo.loc[orderInfo['hosp_dept'].isnull(),'hosp_dept'] = orderInfo["enc_dept"]
orderInfo.drop("enc_dept", axis=1, inplace=True)

In [4]:
OILimited = orderInfo[["MRN", "Visit_id", "antibiotic", "susceptibility", "organism", "description", "hosp_dept", 'Gram']] 
#.drop_duplicates( subset = ('MRN', 'Visit_id', 'antibiotic'))
pd.to_datetime( orderInfo["Order_time"])
OILimited['Visit_id'].astype( str )

orgValueCounts = OILimited['organism'].value_counts()
OILimited = OILimited[OILimited['organism'].isin(orgValueCounts[orgValueCounts > 100].index)]
OILimited['Gram'].replace('NEGATIVE', 0, inplace = True)
OILimited['Gram'].replace('POSITIVE', 1, inplace = True)


OILimited

,MRN,Visit_id,antibiotic,susceptibility,organism,description,hosp_dept,Gram
0,3525540,66008241995,Amikacin,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
1,3525540,66008241995,Ampicillin,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
2,3525540,66008241995,Ampicillin/Sulbactam,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
3,3525540,66008241995,Cefazolin,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
4,3525540,66008241995,Cefepime,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
5,3525540,66008241995,Cefoxitin,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
6,3525540,66008241995,Ceftazidime,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
7,3525540,66008241995,Ceftriaxone,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
8,3525540,66008241995,Cefuroxime,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0
9,3525540,66008241995,Ciprofloxacin,Susceptible,ESCHERICHIA COLI,URINE CULTURE,MON UROLOGY,0


##  Map Susceptibility values into antibiotic cells

In [5]:
#map susceptibility values to numbers
suscmapping={'Susceptible':2,
             'Positive':2,
             'Resistant':-2, 
             'Negative': -2, 
             'Not Susceptible':-2,
             'Intermediate':-1,              
             'Not Defined': -1,         
             'Dose Dependent':-1}

sus = OILimited['susceptibility'].map(suscmapping)

#preprocessing turn antibiotics into columns
antibioticColumns = pd.get_dummies(OILimited['antibiotic'])
antibioticColumns=antibioticColumns.multiply(sus.values, axis=0)
OIEncoded=OILimited.drop('susceptibility', axis=1)
OIEncoded=OIEncoded.drop('antibiotic', axis=1)
OIEncoded=OIEncoded.join(antibioticColumns)

#group by visit_id (visit_id is unique and better suited than MRN) and specimen_comments (sometimes several samples are analyzed at the same time)
OICompressed = OIEncoded.groupby([ 'MRN',
                               'Visit_id',
                               'organism',
                               'description',  
                               'hosp_dept', 
                               'Gram',
                             ]).agg(sum).reset_index()


suscColumns=OICompressed.columns[6:]
suscColumns=suscColumns.insert(0, 'Visit_id')
suscColumns=suscColumns.insert(1, 'organism')

susceptibilities =  OICompressed[suscColumns].copy()
susceptibilities['organism'] = susceptibilities['organism'].astype('category')

susceptibilities.shape

KeyError: 'gram'

In [ ]:
orgColumns = pd.get_dummies(OICompressed['organism'])
OICompressed=OICompressed.drop('organism', axis=1)
OIEncoded=OICompressed.join(orgColumns)
OIEncoded

In [ ]:
#preprocessing turn organisms into columns
OIEncoded['hosp_dept'] = OIEncoded['hosp_dept'].str.split().str[0]
deptColumns = pd.get_dummies(OIEncoded['hosp_dept'])
OIEncoded=OIEncoded.drop('hosp_dept', axis=1)
OIEncoded2=OIEncoded.join(deptColumns)
OIEncoded2.shape

In [ ]:
demoLimited = demo[["MRN", "zip"]]
demoLimited['zip'] = demoLimited['zip'].str.slice( 0, 3 )
OIEncoded3 = OIEncoded2.merge( demoLimited, on = "MRN", how = "outer" )
demoLimited

In [ ]:
#preprocessing turn organisms into columns
descCols = pd.get_dummies(OIEncoded3['description'])
OIEncoded3=OIEncoded3.drop('description', axis=1)
OIEncoded4 = OIEncoded3.join(descCols)
OIEncoded4

In [ ]:
mldataset = OIEncoded4.copy(deep = True)
mldataset.drop_duplicates(inplace = True)

In [ ]:
mldataset = mldataset.fillna(0)

del mldataset['MRN']

mldataset

### Training: Gram,  hospital(2 letter abv), zip code(first 3 numbers)
Target: anti, org

## Taking slices of mldataset By ICD Codes

### Sepsis

In [ ]:
limitedICDCodes = problem_list[['Visit_id', 'ICD10']]

limitedICDCodes['ICD10'] = limitedICDCodes['ICD10'].str.split('.').str[0]

sepsisICD = ['A41', 'R65', 'R78']

sepsisVisits = limitedICDCodes[limitedICDCodes['ICD10'].isin( sepsisICD )]

sepsisVisits.drop_duplicates(inplace = True)

sepsisMLDataset = mldataset[mldataset['Visit_id'].isin(sepsisVisits['Visit_id'].unique())]
sepsisMLDataset

### UTI

In [ ]:
limitedICDCodes = problem_list[['Visit_id', 'ICD10']]
limitedICDCodes['ICD10'] = limitedICDCodes['ICD10'].str.split('.').str[0]

utiICD = ['N39']

utiVisits = limitedICDCodes[limitedICDCodes['ICD10'].isin( utiICD )]

utiVisits.drop_duplicates(inplace = True)

utiMLDataset = mldataset[mldataset['Visit_id'].isin(utiVisits['Visit_id'].unique())]
utiMLDataset

### Pneumonia

In [ ]:
limitedICDCodes = problem_list[['Visit_id', 'ICD10']]
limitedICDCodes['ICD10'] = limitedICDCodes['ICD10'].str.split('.').str[0]

pneumoniaICD = ['J18']

pneumoniaVisits = limitedICDCodes[limitedICDCodes['ICD10'].isin( pneumoniaICD )]

pneumoniaVisits.drop_duplicates(inplace = True)

pneumoniaMLDataset = mldataset[mldataset['Visit_id'].isin(pneumoniaVisits['Visit_id'].unique())]
pneumoniaMLDataset

In [ ]:
%store mldataset
%store sepsisMLDataset
%store utiMLDataset
%store pneumoniaMLDataset